In [3]:
import requests
import pandas as pd
import glob
from pathlib import Path

BASE_DIR = Path().resolve()
CSV_DIR = next((p / "CSV_Categories" for p in [BASE_DIR, *BASE_DIR.parents] if (p / "CSV_Categories").exists()), None)

if CSV_DIR is None:
    raise FileNotFoundError("Impossible de trouver le dossier 'CSV_Categories' dans ce projet.")


In [4]:
# récupérer tous les fichiers du dossier videos et dédupliquer les lignes
files = [
    CSV_DIR / "market_automobile_fr.csv",
    CSV_DIR / "market_jeux_video_fr.csv"
]

df = []
for f in files:
    csv = pd.read_csv(f)
    df.append(csv)
df = pd.concat(df,ignore_index=True)
df.drop_duplicates(subset=['video_id'], keep="first", inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1917 entries, 0 to 1916
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   video_id      1917 non-null   object
 1   title         1917 non-null   object
 2   description   1436 non-null   object
 3   channel       1917 non-null   object
 4   published_at  1917 non-null   object
 5   duration      1917 non-null   object
 6   views         1917 non-null   int64 
 7   likes         1917 non-null   int64 
 8   comments      1917 non-null   int64 
 9   channel_id    1917 non-null   object
 10  category_id   1917 non-null   int64 
 11  language      1868 non-null   object
dtypes: int64(4), object(8)
memory usage: 179.8+ KB


In [5]:
# langues: supprimer les lignes avec valeurs
df.dropna(subset="language",axis=0,inplace=True)

# langues: regrouper fr_fr, be_fr sous 'fr', pareil pour 'en'
df.loc[df['language'].str.contains('^fr'),'language'] = "fr"
df.loc[df['language'].str.contains('^en'),'language'] = "en"

In [6]:
#supprimer autres langues
df['language'].value_counts()

language
fr        670
en        527
hi        282
de         49
es         35
id         30
zxx        27
it         25
bn         24
te         22
ru         21
de-DE      21
ta         15
vi         13
pl         12
ar          8
pa          7
th          7
pt-BR       6
pt-PT       6
tr          5
az          4
es-419      4
ur          4
ml          4
ko          4
es-MX       3
zh          3
ro          3
ja          3
kn          2
nl          2
es-US       2
und         2
sr          2
cs          2
uk          1
sk          1
nl-BE       1
mo          1
or          1
nl-NL       1
gu          1
mr          1
ba          1
no          1
fil         1
bn-IN       1
Name: count, dtype: int64

In [8]:
print(df.columns.tolist())



['video_id', 'title', 'description', 'channel', 'published_at', 'duration', 'views', 'likes', 'comments', 'channel_id', 'category_id', 'language']


In [11]:
# calculer le taux d'engagement pour voir en un coup d'oeil les vidéos ayant reçu le plus d'intéractions
df["Engagement total"] = df["likes"] + df["comments"]
df["Taux d'engagement (%)"] = (df["Engagement total"] / df["views"].replace(0, 1)) * 100

In [12]:
# transofrmer les durées (ici min/sec) en secondes (plus simple pour l'analyse
import re

# 1. Conversion en timedelta (errors='coerce' transforme les formats invalides en NaT)
df['duration_td'] = pd.to_timedelta(df['duration'], errors='coerce')

# 2. Conversion en secondes
# .astype('Int64') permet de garder les NaN si nécessaire sans faire planter le code
df['Durée (s)'] = df['duration_td'].dt.total_seconds().astype('Int64')

# OPTIONNEL : Si vous voulez remplacer les vides par 0 et avoir des entiers classiques
# df['Durée (s)'] = df['duration_td'].dt.total_seconds().fillna(0).astype(int)

print(df[['duration', 'Durée (s)']])

      duration  Durée (s)
0        PT37S         37
1      PT6M13S        373
2        PT19S         19
3         PT8S          8
4        PT37S         37
...        ...        ...
1912   PT1M48S        108
1913     PT21S         21
1914     PT26S         26
1915     PT33S         33
1916  PT45M44S       2744

[1868 rows x 2 columns]


In [13]:
# colonnes avec valeurs nulls
df.isna().sum()

video_id                   0
title                      0
description              473
channel                    0
published_at               0
duration                   0
views                      0
likes                      0
comments                   0
channel_id                 0
category_id                0
language                   0
Engagement total           0
Taux d'engagement (%)      0
duration_td                0
Durée (s)                  0
dtype: int64

In [15]:
clean_name = f.with_name(f.stem + "_clean.csv")

df.to_csv(clean_name, index=False)

